<a href="https://colab.research.google.com/github/ratulKabir/MotorAI_ML_challange/blob/master/ElectronicDevices_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering
In this notebook, clustering is first done using Deep Learning model Autoencoder. Later KMeans is used to perform clustering on the same dataset to compare accuracy.

## Import dataset

In [1]:
from google.colab import drive
from time import time
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans

Using TensorFlow backend.


## Autoencoder


In [0]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img = Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

## Import dataset


In [3]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/ElectricDevices.csv', header=-1)

In [0]:
x = df.iloc[:, 1:] # spliting the label from the original dataset to
y = df.iloc[:, 0]  # feed the model

In [0]:
# number of clusters
n_clusters = len(np.unique(y))

In [0]:
dims = [x.shape[1], 500, 500, 2000, 7]
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_epochs = 100
batch_size = 256
save_dir = '/content/drive/My Drive/Colab Notebooks/input' #directory to save the weights

In [0]:
autoencoder, encoder = autoencoder(dims, init=init)

#### Pre-train autoencoder

In [11]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)

Epoch 1/100
7711/7711 [==============================] - 2s 312us/step - loss: 0.8498
Epoch 2/100
7711/7711 [==============================] - 3s 381us/step - loss: 0.8195
Epoch 3/100
7711/7711 [==============================] - 3s 377us/step - loss: 0.7969
Epoch 4/100
7711/7711 [==============================] - 3s 379us/step - loss: 0.8157
Epoch 5/100
7711/7711 [==============================] - 3s 375us/step - loss: 0.7635
Epoch 6/100
7711/7711 [==============================] - 3s 377us/step - loss: 0.7220
Epoch 7/100
7711/7711 [==============================] - 3s 375us/step - loss: 0.6909
Epoch 8/100
7711/7711 [==============================] - 3s 370us/step - loss: 0.6573
Epoch 9/100
7711/7711 [==============================] - 3s 373us/step - loss: 0.6375
Epoch 10/100
7711/7711 [==============================] - 3s 372us/step - loss: 0.6230
Epoch 11/100
7711/7711 [==============================] - 3s 370us/step - loss: 0.6021
Epoch 12/100
7711/7711 [============================

In [0]:
autoencoder.save_weights(save_dir + '/ae_weights.h5') # save weights

In [0]:
autoencoder.load_weights(save_dir + '/ae_weights.h5') # load weights


## Clustering using Autoencoder
This is the clustering layer that uses the pre-trained weights of the encoder. 

In [0]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [0]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)

In [0]:
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')


## Initialize cluster centers
Only the cluster centers are initialized using KMeans. The actual clustering is done by the clustering layer previously created.

In [0]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20) # initialize cluster centers
y_pred = kmeans.fit_predict(encoder.predict(x))

In [0]:
y_pred_last = np.copy(y_pred)

In [0]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])


In [0]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [0]:
loss = 0
index = 0
maxiter = 50
update_interval = 140
index_array = np.arange(x.shape[0])
#tol = 0.001 # tolerance threshold to stop training

## Define accuracy
In the metrics class a function acc() is defined. It takes as input the real labels and the predicted values and computes the accuracy.

In [0]:
class metrics:
  import numpy as np
  from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

  nmi = normalized_mutual_info_score
  ari = adjusted_rand_score

 
  def acc(y_true, y_predict):
      """
      Calculate clustering accuracy. Require scikit-learn installed
      # Arguments
          y: true labels, numpy.array with shape `(n_samples,)`
          y_pred: predicted labels, numpy.array with shape `(n_samples,)`
      # Return
          accuracy, in [0,1]
      """
      y_true = y_true.astype(np.int64)
      assert y_predict.size == y_true.size
      D = max(y_predict.max(), y_true.max()) + 1
      w = np.zeros((D, D), dtype=np.int64)
      for i in range(y_predict.size):
          w[y_predict[i], y_true[i]] += 1
      from sklearn.utils.linear_assignment_ import linear_assignment
      ind = linear_assignment(w.max() - w)
      return sum([w[i, j] for i, j in ind]) * 1.0 / y_predict.size

## Traning

In [0]:
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(metrics.acc(y, y_pred), 5)
            nmi = np.round(metrics.nmi(y, y_pred), 5)
            ari = np.round(metrics.ari(y, y_pred), 5)
            loss = np.round(loss, 5)
        # check stop criterion - model convergence
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/DEC_model_final.h5')

In [0]:
model.load_weights(save_dir + '/DEC_model_final.h5')


## Final evaluation

In [25]:
# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    acc = np.round(metrics.acc(y, y_pred), 5)
    nmi = np.round(metrics.nmi(y, y_pred), 5)
    ari = np.round(metrics.ari(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('Acc = %.5f, nmi = %.5f, ari = %.5f' % (acc, nmi, ari), ' ; loss=', loss)

Acc = 0.39541, nmi = 0.18594, ari = 0.19198  ; loss= 0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


## Clustering using KMeans

In [0]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20, n_jobs=4)
y_pred_kmeans = kmeans.fit_predict(x)

In [39]:
metrics.acc(y, y_pred_kmeans)


0.37906886266372714

## Final comparison
Accuracy using Autoencoder = 0.39541<br>
Accuracy using KMeans = 0.37906
